In [9]:
# --- Para el proceso de datos y visualización ---
import numpy as np
import pandas as pd
import json 
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.mode.chained_assignment = None  # default='warn'
from google.cloud import storage



# --- Dependencias de Vertex AI ---
import vertexai                                              # Importa el módulo principal de Vertex AI.
from vertexai import init                                    # Inicializa Vertex AI con las credenciales y configuraciones necesarias.
from vertexai.vision_models import MultiModalEmbeddingModel  # Importa el modelo de embeddings multimodales de Vertex AI para procesar imágenes y videos.
from vertexai.vision_models import Video                     # Clase para manejar archivos de video en Vertex AI.
from vertexai.vision_models import VideoSegmentConfig        # Configuración para segmentar videos al gener
from google.cloud.aiplatform.matching_engine import MatchingEngineIndexEndpoint 
from vertexai.vision_models import Image

# --- Dependencias para poder visualizar ---
from IPython.display import Video as MVideo                  # Permite mostrar videos directamente en celdas de Jupyter Notebook.
from IPython.display import HTML                             # Permite mostrar contenido HTML en celdas de Jupyter Notebook.
from IPython.display import Image as ImageByte               # Permite mostrar imágenes en el notebook (renombrado como ImageByte para evitar conflictos de nombres).
from IPython.display import display                          # Función general para mostrar objetos en el notebook.
from sklearn.metrics.pairwise import cosine_similarity       # Función para calcular la similitud coseno entre vectores, útil para comparar embeddings.
from vertexai.vision_models import Video, VideoSegmentConfig

In [11]:
# --- Cargamos el modelo de embeddings multimodales ---
mm_embendding_model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")

In [12]:
# --- Cargamos las credewntiales de Vertex AI ---

PROJECT_ID = "dauntless-drive-462416-q3"
LOCATION = "us-central1"


# --- Inicializamos Vertex AI ---
init(project = PROJECT_ID, location = LOCATION)

In [14]:
import os
import subprocess
from google.cloud import storage

def fragmentar_video_local_ffmpeg(
    ruta_video_local: str,
    gcs_bucket_salida: str,
    duracion_segmento: int,
    carpeta_tmp: str = "/tmp"
):
    """
    Fragmenta un video local en segmentos de X segundos usando ffmpeg y sube SOLO los fragmentos a un bucket de GCS.
    """
    nombre_base = os.path.splitext(os.path.basename(ruta_video_local))[0]
    storage_client = storage.Client()
    bucket_out = storage_client.bucket(gcs_bucket_salida)

    # Fragmentar con ffmpeg
    output_pattern = os.path.join(carpeta_tmp, f"{nombre_base}_segment_%03d.mkv")
    cmd = [
        "ffmpeg",
        "-i", ruta_video_local,
        "-c", "copy",
        "-map", "0",
        "-f", "segment",
        "-segment_time", str(duracion_segmento),
        output_pattern
    ]
    print("Fragmentando video con ffmpeg...")
    subprocess.run(cmd, check=True)

    # Subir SOLO los fragmentos al bucket de salida
    fragmentos = sorted([
        f for f in os.listdir(carpeta_tmp)
        if f.startswith(f"{nombre_base}_segment_") and f.endswith(".mkv")
    ])
    print(f"Subiendo {len(fragmentos)} fragmentos a gs://{gcs_bucket_salida}/ ...")
    for frag in fragmentos:
        local_path = os.path.join(carpeta_tmp, frag)
        blob_dest = f"{nombre_base}/{frag}"
        bucket_out.blob(blob_dest).upload_from_filename(local_path)
        print(f"  ✅ Subido: gs://{gcs_bucket_salida}/{blob_dest}")

    # Limpiar archivos temporales
    for frag in fragmentos:
        os.remove(os.path.join(carpeta_tmp, frag))
    print("¡Listo!")

# Ejemplo de uso:
fragmentar_video_local_ffmpeg(
    ruta_video_local="/Users/wivboost/Downloads/azteca7_recording.mkv",
    gcs_bucket_salida="fragmentaciones",
    duracion_segmento=20  # segundos
)

Fragmentando video con ffmpeg...


ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.0.13.3)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_3 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

Subiendo 404 fragmentos a gs://fragmentaciones/ ...
  ✅ Subido: gs://fragmentaciones/azteca7_recording/azteca7_recording_segment_000.mkv
  ✅ Subido: gs://fragmentaciones/azteca7_recording/azteca7_recording_segment_001.mkv
  ✅ Subido: gs://fragmentaciones/azteca7_recording/azteca7_recording_segment_002.mkv
  ✅ Subido: gs://fragmentaciones/azteca7_recording/azteca7_recording_segment_003.mkv
  ✅ Subido: gs://fragmentaciones/azteca7_recording/azteca7_recording_segment_004.mkv
  ✅ Subido: gs://fragmentaciones/azteca7_recording/azteca7_recording_segment_005.mkv
  ✅ Subido: gs://fragmentaciones/azteca7_recording/azteca7_recording_segment_006.mkv
  ✅ Subido: gs://fragmentaciones/azteca7_recording/azteca7_recording_segment_007.mkv
  ✅ Subido: gs://fragmentaciones/azteca7_recording/azteca7_recording_segment_008.mkv
  ✅ Subido: gs://fragmentaciones/azteca7_recording/azteca7_recording_segment_009.mkv
  ✅ Subido: gs://fragmentaciones/azteca7_recording/azteca7_recording_segment_010.mkv
  ✅ Subido: g

In [16]:
def get_video_embedding(ruta_video: str) -> list: 
    
    """
    Genera un embedding para un video dado.

    Args:
        ruta_video (str): Ruta al archivo de video.

    Returns:
        list: Embedding del video.
    """
    # Cargamos el video desde la ruta proporcionada
    video = Video.load_from_file(ruta_video)
    
    # Genera el embedding del video utilizando el modelo de embeddings multimodales
    embedding = mm_embendding_model.get_embeddings(video = video, 
                                                   video_segment_config = VideoSegmentConfig(interval_sec=4) # Configura el segmento del video para generar embeddings cada 4 segundos.
                                                  )
    
    return [video_emb.embedding for video_emb in embedding.video_embeddings]  # Retorna una lista de embeddings para cada segmento del video.



In [17]:
def guardar_embeddings_en_gcs_json(
    project_id: str,
    bucket_name: str,
    blob_name: str,
    ids: list[str],
    embeddings: list[list[float]]
):
    """
    Guarda los embeddings en un solo archivo JSON en GCS.
    """
    import json
    print(f"Conectando al bucket '{bucket_name}'...")
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    # Prepara los datos como una lista de diccionarios
    data = [{"id": i, "embedding": emb} for i, emb in zip(ids, embeddings)]

    print(f"Escribiendo {len(ids)} embeddings en formato JSON...")
    # Escribe el archivo JSON en GCS
    blob.upload_from_string(
        data=json.dumps(data, ensure_ascii=False, indent=2),
        content_type="application/json"
    )
    print(f"¡Éxito! Archivo '{blob_name}' subido correctamente a 'gs://{bucket_name}/{blob_name}'.")

In [18]:
import os
from google.cloud import storage
import json

def procesar_fragmentos_y_guardar_embeddings(
    project_id: str,
    bucket_name: str,
    carpeta_fragmentos: str,
    carpeta_tmp: str = "/tmp"
):
    """
    Descarga cada fragmento de video de GCS, genera embeddings y los guarda en el mismo bucket
    en una carpeta con el sufijo '_embeddings'.
    """
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.bucket(bucket_name)
    carpeta_embeddings = carpeta_fragmentos + "_embeddings"

    blobs = list(storage_client.list_blobs(bucket_name, prefix=carpeta_fragmentos + "/"))
    fragmentos = [blob.name for blob in blobs if blob.name.endswith(".mkv")]

    print(f"Procesando {len(fragmentos)} fragmentos...")

    for frag_blob_name in fragmentos:
        frag_local = os.path.join(carpeta_tmp, os.path.basename(frag_blob_name))
        bucket.blob(frag_blob_name).download_to_filename(frag_local)
        print(f"Descargado: {frag_blob_name}")

        embeddings = get_video_embedding(frag_local)
        ids = [f"{os.path.basename(frag_blob_name)}_seg{i}" for i in range(len(embeddings))]

        emb_blob_name = f"{carpeta_embeddings}/embeddings_{os.path.basename(frag_blob_name)}.jsonl"
        guardar_embeddings_en_gcs_json(
            project_id=project_id,
            bucket_name=bucket_name,
            blob_name=emb_blob_name,
            ids=ids,
            embeddings=embeddings
        )
        print(f"  ✅ Embeddings guardados: gs://{bucket_name}/{emb_blob_name}")

        os.remove(frag_local)
    print("¡Listo!")

In [19]:
def procesar_fragmentos_y_guardar_embeddings(
    project_id: str,
    bucket_name: str,
    carpeta_fragmentos: str,
    carpeta_tmp: str = "/tmp"
):
    """
    Descarga cada fragmento de video de GCS, genera embeddings y los guarda en el mismo bucket
    en una carpeta con el sufijo '_embeddings', en formato JSON.
    """
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.bucket(bucket_name)
    carpeta_embeddings = carpeta_fragmentos + "_embeddings"

    blobs = list(storage_client.list_blobs(bucket_name, prefix=carpeta_fragmentos + "/"))
    fragmentos = [blob.name for blob in blobs if blob.name.endswith(".mkv")]

    print(f"Procesando {len(fragmentos)} fragmentos...")

    for frag_blob_name in fragmentos:
        frag_local = os.path.join(carpeta_tmp, os.path.basename(frag_blob_name))
        bucket.blob(frag_blob_name).download_to_filename(frag_local)
        print(f"Descargado: {frag_blob_name}")

        embeddings = get_video_embedding(frag_local)
        ids = [f"{os.path.basename(frag_blob_name)}_seg{i}" for i in range(len(embeddings))]

        emb_blob_name = f"{carpeta_embeddings}/embeddings_{os.path.basename(frag_blob_name)}.json"
        guardar_embeddings_en_gcs_json(
            project_id=project_id,
            bucket_name=bucket_name,
            blob_name=emb_blob_name,
            ids=ids,
            embeddings=embeddings
        )
        print(f"  ✅ Embeddings guardados: gs://{bucket_name}/{emb_blob_name}")

        os.remove(frag_local)
    print("¡Listo!")

# Ejemplo de uso:
procesar_fragmentos_y_guardar_embeddings(
    project_id="dauntless-drive-462416-q3",
    bucket_name="fragmentaciones",
    carpeta_fragmentos="azteca7_recording"
)

Procesando 404 fragmentos...
Descargado: azteca7_recording/azteca7_recording_segment_000.mkv
Conectando al bucket 'fragmentaciones'...
Escribiendo 6 embeddings en formato JSON...
¡Éxito! Archivo 'azteca7_recording_embeddings/embeddings_azteca7_recording_segment_000.mkv.json' subido correctamente a 'gs://fragmentaciones/azteca7_recording_embeddings/embeddings_azteca7_recording_segment_000.mkv.json'.
  ✅ Embeddings guardados: gs://fragmentaciones/azteca7_recording_embeddings/embeddings_azteca7_recording_segment_000.mkv.json
Descargado: azteca7_recording/azteca7_recording_segment_001.mkv
Conectando al bucket 'fragmentaciones'...
Escribiendo 5 embeddings en formato JSON...
¡Éxito! Archivo 'azteca7_recording_embeddings/embeddings_azteca7_recording_segment_001.mkv.json' subido correctamente a 'gs://fragmentaciones/azteca7_recording_embeddings/embeddings_azteca7_recording_segment_001.mkv.json'.
  ✅ Embeddings guardados: gs://fragmentaciones/azteca7_recording_embeddings/embeddings_azteca7_reco